<a href="https://colab.research.google.com/github/moraesleonardo/Associacoes_para_Machine_Learning/blob/main/Associa%C3%A7%C3%B5es_para_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Associações

## Exemplo

### Base de produtos bancários

Referência: <http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/>

### Importando bibliotecas

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### Lendo a base

In [ ]:
cesta_ = pd.read_csv('cesta_produtos.csv')

In [ ]:
cesta_

### Preparando a base

# Removendo entradas de clientes com apenas um produto
cesta_['soma'] = cesta_.count(axis=1)
cesta_ = cesta_[cesta_['soma'] != 1]
cesta_.drop('soma', axis=1, inplace=True)

In [ ]:
# Convertendo valores das células para 0s ou 1s.
cesta_ = cesta_.notnull().astype('int')

In [ ]:
cesta_

### Aplicando o algoritmo Apriori

In [ ]:
# Gerando conjuntos de itens frequentes
frequent_itemsets_ = apriori(cesta_, min_support=0.01, use_colnames=True)

In [ ]:
# Incluindo a quantidade de itens por entrada
frequent_itemsets_['length'] = frequent_itemsets_['itemsets'].apply(lambda x: len(x))

In [ ]:
frequent_itemsets_

In [ ]:
# Gerando regras
rules_ = association_rules(frequent_itemsets_, metric="lift", min_threshold=1)

In [ ]:
# Convertendo o formato frozenset para lista comum
frequent_itemsets_['itemsets'] = frequent_itemsets_['itemsets'].apply(lambda x: list(x)).astype('unicode')
rules_['antecedents'] = rules_['antecedents'].apply(lambda x: list(x)).astype('unicode')
rules_['consequents'] = rules_['consequents'].apply(lambda x: list(x)).astype('unicode')

In [ ]:
rules_

### Ajustando a base

In [ ]:
# Removendo duplicatas - Resultado APRIORI
# Ordenando por Confiança
rules_.sort_values('confidence', ascending=False, inplace=True)

In [ ]:
# Removendo duplicatas por Suporte e Lift, mantendo a primeira ocorrência (maior confiança)
rules_.drop_duplicates(['support','lift'], keep='first', inplace=True)

### Visualizações

In [ ]:
# Ordenando resultados pela métrica Suporte
frequent_itemsets_.sort_values('support', ascending=False, inplace=True)

In [ ]:
# Mantendo apenas as carteiras de maior suporte
frequent_itemsets_ = frequent_itemsets_.head(15)

In [ ]:
# Criando a visualização
fig, ax1 = plt.subplots(figsize=(9, 15))
sns.barplot(x='support', y='itemsets', data=frequent_itemsets_)
ax1.set(ylabel='Conjuntos de itens')
ax1.set(xlabel='Suporte')
plt.title('Conjuntos de itens - Produtos (Top-15 Suporte)')
plt.tight_layout()
fig.savefig('Conjuntos_de_itens_Carteiras_Suporte.png')

In [ ]:
# Função para geração de visualização Confiança x Lift
def visualiza_regras_conf_lift(df, tipo):
    df.sort_values(['lift','confidence'], ascending=[False,False], inplace=True)
    fig, ax1 = plt.subplots(figsize=(10, 10))
    sns.scatterplot(x='consequents', y='antecedents', data=df, size='confidence',
                    palette='icefire', edgecolors='black',
                    hue='lift', legend=True, alpha=0.8, sizes=(10,1000))
    ax1.tick_params(axis='x', labelrotation=90)
    ax1.set(ylabel='LHS (Antecedentes)')
    ax1.set(xlabel='RHS (Consequentes)')
    plt.title(tipo + ' (Regras) - Relação Confiança x Lift')
    plt.grid(alpha=0.2)
    plt.tight_layout()
    fig.savefig('Regras - Confiança x Lift.png')

In [ ]:
# Visualização Regras - Confiança x Lift
# Filtrando para apresentar uma quantidade menor de resultados
rules_1 = rules_[rules_['lift'] > 7.6]

In [ ]:
# Apresentando a visualização
visualiza_regras_conf_lift(rules_1, 'Produtos')

### Visualizações utilizando grafos (NetworkX)

In [ ]:
import networkx as nx
from bokeh.io import show, save
from bokeh.models import Range1d, Circle, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

# Definindo o que será apresentado
fig, ax=plt.subplots(figsize=(20,15))
GA = nx.from_pandas_edgelist(rules_1, source='antecedents', target='consequents')
nx.draw(GA, with_labels=True, font_size=10)

# Definindo um título
title = 'Relações entre produtos'

# Definindo o que será apresentado ao passar o mouse pela informação
HOVER_TOOLTIPS = [('Antecedente', '@index')]

# Criando a visualização
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-15.1, 15.1), y_range=Range1d(-15.1, 15.1), title=title)

# Criando um grafo da rede com o layout 'spring'
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(GA, nx.spring_layout, scale=15, center=(0, 0))

# Configurando cor e tamenaho do nó
network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')
# Configurando opacidade etamanho do vértice
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
# Adicionando o grafo da rede na visualização
plot.renderers.append(network_graph)

#show(plot)
save(plot, filename=f"{title}.html")